In [ ]:
#pip install transformers

In [119]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter

import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)
import os

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [2]:
df = pd.read_pickle('../../data/dataset.p')

In [3]:
texts = df.text_clean
#add markers
texts = ["[CLS] " + text.lower() + " [SEP]" for text in texts]
#remove NUM
texts = [text.replace('num','') for text in texts]

In [31]:
#tokenize
tokenized_text = [tokenizer.tokenize(text) for text in texts]

def cut_text(tokens):
    if len(tokens)>=500:
        tokens = tokens[0:499] + ['[SEP]']
    return tokens

tokenized_text = list(map(cut_text,tokenized_text))

In [32]:
indexed_tokens = [tokenizer.convert_tokens_to_ids(text) for text in tokenized_text]

segments_ids = []
for i in range(len(indexed_tokens)):
    segments_id = [i] * len(indexed_tokens[i]) # maybe change [i] for i
    segments_ids.append(segments_id)



[11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703, 11703]         1
[15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 15521, 1

In [33]:
len(segments_ids)

22151

In [34]:
#to tensor
tokens_tensor = [torch.tensor([t]) for t in indexed_tokens]
segments_tensors = [torch.tensor([t]) for t in segments_ids]

In [19]:
#load pretrained model
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [55]:
def sentence_embedding(tokens_tensor,segments_tensors):
    #Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():

        outputs = model(tokens_tensor, segments_tensors)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][0]
        sentence_embedding = torch.mean(token_vecs, dim=0)
        return sentence_embedding

In [84]:
sentences_embeddings = [sentence_embedding(x,y) for x,y in zip(tokens_tensor, segments_tensors)]

In [95]:
semb = sentences_embeddings.copy()

In [102]:
semb = [s.detach().numpy().tolist() for s in sentences_embeddings]

In [106]:
df['bert_embedding'] = semb

In [108]:

df.to_pickle('../../data/dataset.p')